In [26]:
from collections import defaultdict, Counter
from copy import deepcopy
import logging as log

In [27]:
args = p.parse_args()
if args.verbose:
    log.basicConfig(format="%(levelname)s: %(message)s", level=log.DEBUG)
    log.info("Verbose output.")
else:
    log.basicConfig(format="%(levelname)s: %(message)s")

log.info("This should be verbose.")
log.warning("This is a warning.")
log.error("This is an error.")

NameError: name 'p' is not defined

In [84]:
EXAMPLES_DIR = "./"

In [85]:
ncolors2colors = {2:['R', 'G'], 3:['R', 'G', 'B'], 4:['R', 'G', 'B', 'Y']}
clr2color = {'R':'Red', 'G':'Green', 'Y':'Yellow', 'B':'Blue'}

In [86]:
def read_input(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()
    return lines

In [87]:
class DAG:
    def __init__(self, adj_list=None, nodes=None):
        self.adj_list = adj_list if adj_list else defaultdict(list) # {'a': ['a1', 'a2', 'a3'], ...}
        self.nodes = nodes
    
    def create_node_graph(self):
        for node in self.nodes:
            if node not in self.adj_list:
                self.adj_list[node] = []
        for node in self.adj_list:
            for child in self.adj_list[node]:
                self._add_edge(src=node, dest=child)
        return dict(sorted(dict(sorted(self.adj_list.items(), key=lambda x:x[1], reverse=False)).items(), key=lambda x: x[0], reverse=False))
                
                
    def _add_edge(self, src, dest):
        if dest == '':
            self.adj_list[src] = []
            return
        if dest not in self.adj_list[src]:
            self.adj_list[src].append(dest)
        if src not in self.adj_list[dest]:
            self.adj_list[dest].append(src)

In [88]:
def trim_label_name(name):
    return name.strip()

In [95]:
def is_label_name_valid(name):
    if len(name) > 32: return False
    if len([i for i in name if i in [':', '[', ']', ',']])>0:
        return False
    return True

In [102]:
is_label_name_valid(name='aaaaaaaaa,[a')

False

In [97]:
def parse_input(filename):
    lines = read_input(filename=filename)
    lines = [l.strip('\n') for l in lines if l != '\n']  #remove extra spaces from input
        
    adj_list = dict()
    labels = set()
    
    for line in lines:
        if ":" in line:
            src, children = line.split(": ")[0], [c.strip() for c in line.split(": ")[-1].lstrip('[').rstrip(']').split(",")]
            src = trim_label_name(src)
            children = [trim_label_name(c) for c in children]
            if is_label_name_valid(src) and ([is_label_name_valid(c) for c in children] == [True for c in children]):
                adj_list[src] = children
                labels.add(src)
                labels.update(children)
            else:
                print("Invalid label names, try again")
                exit()
            
    # remove empty items from set
    labels.discard('')
    
    return adj_list, labels

In [90]:
def graph_constraints(adj_list, ncolor):
    colors = ncolors2colors[ncolor]
    cnf_clauses = []
    for node in adj_list:
        # Constraint 1: At least one color 
        constraint1 = []
        for color in colors:
            atom = f'{node}_{color}'
            constraint1.append(atom)
        
        # Constrain 2: No adjacent same colors rhs for every edge, distinct clause for each color
        constraint2 = []
        for cons in constraint1:
            for child in sorted(adj_list[node]):
                constraint2.append((f'!{cons} !{child}_{cons[-1]}'))
        
        # Add constraints of a node to cnf clauses for entire graph
        cnf_clauses.extend([" ".join(constraint1)])
        cnf_clauses.extend(constraint2)
    
    return cnf_clauses

In [91]:
def check_missing_clauses(computed_clauses, true_clauses):
    comp_ = set(tuple(row) for row in sorted(list([clause.split(', ') for clause in computed_clauses])))
    true_ = set(tuple(row) for row in sorted(list([clause.split(', ') for clause in true_clauses])))
    
    print(f"Clauses in true set: {len(true_)}, Clauses. in computed set: {len(comp_)}")
    print(f"Missing clauses: {true_.difference(comp_)}")
    print(f"Extra clauses computed: {comp_.difference(true_)}")

In [92]:
def get_true_clauses(filename):
    lines = read_input(filename=filename)
    true_clauses = [l.strip('\n') for l in lines if l != '\n']  #remove extra spaces from input
    return true_clauses

In [94]:
FILE = "us48.txt"
ncolor = 4
adj_list_1, nodes = parse_input(filename=f"{EXAMPLES_DIR}/{FILE}")
al = DAG(adj_list=adj_list_1, nodes=nodes).create_node_graph()
computed_clauses = graph_constraints(adj_list=al, ncolor=ncolor)
true_clauses = get_true_clauses(filename=f"{EXAMPLES_DIR}/{FILE}.{ncolor}.dp")
check_missing_clauses(true_clauses=true_clauses, computed_clauses=computed_clauses)

Clauses in true set: 921, Clauses. in computed set: 921
Missing clauses: set()
Extra clauses computed: set()


In [12]:
computed_clauses

['1_R 1_G',
 '!1_R !2_R',
 '!1_R !4_R',
 '!1_G !2_G',
 '!1_G !4_G',
 '2_R 2_G',
 '!2_R !1_R',
 '!2_R !3_R',
 '!2_G !1_G',
 '!2_G !3_G',
 '3_R 3_G',
 '!3_R !2_R',
 '!3_R !4_R',
 '!3_G !2_G',
 '!3_G !4_G',
 '4_R 4_G',
 '!4_R !1_R',
 '!4_R !3_R',
 '!4_G !1_G',
 '!4_G !3_G']

In [13]:
def get_atoms_and_literals_from_cnf(cnf):
    """
    Atoms: literals without negation
    Literals: Atoms with/without negation
    """
    l = []
    for i in cnf:
        l.extend(i.split(' '))
    atoms = sorted(list(set([i.strip('!') for i in l])))
    literals = list(set([i for i in l]))
    literals_count = dict(Counter(l))
    return atoms, literals, literals_count

In [14]:
def propagate(atom, cnf, values):
    """
    Propagate the effect of assigning atom A to be value V.
    Delete every clause where A appears with sign V
    Delete every literal where A appears with sign not V.
    """
    cnf_ = []
    for clause in cnf:
        clause_literals = clause.split(' ')
        # delete C from S
        if (atom in clause_literals and values[atom]==True) or (f'!{atom}' in clause_literals and values[atom]==False):
            pass
        # delete A from C
        elif atom in clause_literals and values[atom]==False:
            clause_ = ' '.join([i for i in clause.split(' ') if i!=atom])
            cnf_.append(clause_)
        # delete ~A from C
        elif f'!{atom}' in clause_literals and values[atom]==True:
            clause_ = ' '.join([i for i in clause.split(' ') if i!=f'!{atom}'])
            cnf_.append(clause_)
        else:
            cnf_.append(clause)
    return cnf_


def obvious_assign(literal, values):
    """
    Given a literal L with atom A, make V[A] the sign indicated by L.
    """
    if '!' in literal:
        atom = get_atom(literal)
        values[atom] = False
    else:
        atom = literal
        values[atom] = True
    # print(f"assigned atom {atom} as {values[atom]} through obvious assign")
    return values


def is_literal_in_cnf(literal, cnf):
    """
    Gives clause indices where a particular literal is present in cnf clause list
    """
    present = []
    for idx, clause in enumerate(cnf):
        if literal in clause:
            present.append(idx)
    return present


def get_atom(literal):
    """
    Returns atom of the literal
    """
    if '!' in literal:
        return literal[1:]
    else:
        return literal

In [15]:
def get_next_pureliteral(cnf):
    # print(f"Retrieving next pure literal from cnf ...")
    _, literals, literals_count = get_atoms_and_literals_from_cnf(cnf)
    for literal in literals:
        # check if negation of literal in remaining clauses
        if literal[0]=='!':
            literal_negation = literal[1:]
        else:
            literal_negation = '!' + literal
        if len(is_literal_in_cnf(literal=literal_negation, cnf=cnf)) == 0:
            # print(f"Pure literal {literal} found. Negation of this literal is not present in remaining clauses")
            return literal
    # print(f"No pure literal exit, continuing ...")
    return None


def get_next_clause_with_single_occurrence_literal(cnf):
    # print(f"Retrieving single literal clause from cnf ...")
    for clause in cnf:
        literals = clause.split(' ')
        if len(literals) == 1:
            # print(f"Clause with single literal: {literals[0]} found")
            return literals[0]
    # print(f"No clause with single literal exist, continuing ...")
    return None

In [16]:
def delete_pureliteral_from_cnf(literal, cnf):
    clauses_with_pureliteral = []
    for clause in cnf:
        if literal in clause:
            clauses_with_pureliteral.append(clause)
    cnf_ = [clause for clause in cnf if clause not in clauses_with_pureliteral]
    return cnf_

In [17]:
def check_empty_clause(cnf):
    for clause in cnf:
        if len(clause)==0:
            return None
    return ''

In [40]:
def dpl(atoms, cnf, values):
    while(True):
        if len(cnf) == 0:
            print(f"Assigning remaining unbounded atoms as False: {values}")
            for a in atoms:
                if values[a] == 'unbound':
                    print(f"Unbounded {a}, assigning False")
                    values[a] = False
            return values
        
        elif check_empty_clause(cnf) is None:
            return None
        
        elif get_next_pureliteral(cnf) is not None:
            pureliteral = get_next_pureliteral(cnf)
            values = obvious_assign(literal=pureliteral, values=values)
            cnf = delete_pureliteral_from_cnf(literal=pureliteral, cnf=cnf)
            print(f"easy case: pure literal {pureliteral}={values[get_atom(pureliteral)]}")
        
        elif get_next_clause_with_single_occurrence_literal(cnf) is not None:
            literal = get_next_clause_with_single_occurrence_literal(cnf)
            values = obvious_assign(literal=literal, values=values)
            cnf = propagate(get_atom(literal), cnf, values)
            print(f"easy case: unit literal {literal}")
            
        else:
            break
    

    unbound_atom = [k for k in values.keys() if values[k]=='unbound'][0]
    print(f"hard case: guess {unbound_atom}=True")
    values[unbound_atom] = True
    cnf1 = deepcopy(cnf)
    cnf1 = propagate(unbound_atom, cnf1, values)
    values_new = dpl(atoms, cnf1, values)
    if (values_new != None):
        return values_new
    
    print(f"contradiction: backtrack guess {unbound_atom}=False")
    values[unbound_atom] = False
    cnf1 = propagate(unbound_atom, cnf, values)
    return dpl(atoms, cnf1, values)
            

In [41]:
def dp(atoms, cnf):
    values = {k:'unbound' for k in atoms}
    return dpl(atoms, cnf, values)

In [42]:
# atoms = ['C', 'A', 'B']
# cnf = ['A B C', 'A !B C', '!A B !C', '!C']
cnf = computed_clauses
atoms, literals, literals_count = get_atoms_and_literals_from_cnf(cnf)
print(f"Atoms: ", atoms)
print(f"Literals: ", literals)
print(f"CNF: ", cnf)

Atoms:  ['1_G', '1_R', '2_G', '2_R', '3_G', '3_R', '4_G', '4_R']
Literals:  ['!1_G', '2_G', '!4_G', '!4_R', '!1_R', '2_R', '1_G', '!3_G', '!2_R', '!2_G', '3_R', '3_G', '1_R', '4_R', '4_G', '!3_R']
CNF:  ['1_R 1_G', '!1_R !2_R', '!1_R !4_R', '!1_G !2_G', '!1_G !4_G', '2_R 2_G', '!2_R !1_R', '!2_R !3_R', '!2_G !1_G', '!2_G !3_G', '3_R 3_G', '!3_R !2_R', '!3_R !4_R', '!3_G !2_G', '!3_G !4_G', '4_R 4_G', '!4_R !1_R', '!4_R !3_R', '!4_G !1_G', '!4_G !3_G']


In [43]:
dp(atoms, cnf)


hard case: guess 1_G=True
easy case: unit literal !2_G
{'1_G': True, '1_R': 'unbound', '2_G': False, '2_R': 'unbound', '3_G': 'unbound', '3_R': 'unbound', '4_G': 'unbound', '4_R': 'unbound'}
easy case: unit literal !4_G
{'1_G': True, '1_R': 'unbound', '2_G': False, '2_R': 'unbound', '3_G': 'unbound', '3_R': 'unbound', '4_G': False, '4_R': 'unbound'}
easy case: pure literal 3_G=True
{'1_G': True, '1_R': 'unbound', '2_G': False, '2_R': 'unbound', '3_G': True, '3_R': 'unbound', '4_G': False, '4_R': 'unbound'}
easy case: unit literal 2_R
{'1_G': True, '1_R': 'unbound', '2_G': False, '2_R': True, '3_G': True, '3_R': 'unbound', '4_G': False, '4_R': 'unbound'}
easy case: unit literal !1_R
{'1_G': True, '1_R': False, '2_G': False, '2_R': True, '3_G': True, '3_R': 'unbound', '4_G': False, '4_R': 'unbound'}
easy case: unit literal !3_R
{'1_G': True, '1_R': False, '2_G': False, '2_R': True, '3_G': True, '3_R': False, '4_G': False, '4_R': 'unbound'}
easy case: pure literal 4_R=True
{'1_G': True, '

{'1_G': True,
 '1_R': False,
 '2_G': False,
 '2_R': True,
 '3_G': True,
 '3_R': False,
 '4_G': False,
 '4_R': True}

In [39]:
assignments = {
    '1_G': True,
    '1_R': False,
    '2_G': False,
    '2_R': True,
    '3_G': True,
    '3_R': False,
    '4_G': False,
    '4_R': True
}
convert_back(assignments)

1 Green
2 Red
3 Green
4 Red


{'1': 'Green', '2': 'Red', '3': 'Green', '4': 'Red'}

In [38]:
def convert_back(assignments):
    true_assignments = [i for i in assignments if assignments[i] is True]
    solution = {}
    for assignment in true_assignments:
        label, value = assignment.split('_')[0], clr2color[assignment.split('_')[-1]]
        solution[label]=value
    return solution

In [44]:
def write2file(l, filename):
    # open file in write mode
    with open(filename, 'w') as fp:
        for item in l:
            # write each item on a new line
            fp.write("%s\n" % item)

In [45]:
# list of names
names = ['Jessa', 'Eric', 'Bob']

write2file(names, 'names.txt')


In [50]:
constraint1 = ['a', 'b', 'c']
cons_set = []
for cons in constraint1:
    cons_set.append((cons, [c for c in constraint1 if c != cons]))

In [51]:
cons_set

[('a', ['b', 'c']), ('b', ['a', 'c']), ('c', ['a', 'b'])]

In [71]:
def get_next_pureliteral(cnf):
    print(f"Retrieving next pure literal from cnf ...")
    # Get counts of literals in the cnf
    l = []
    for i in cnf:
        l.extend(i.split(' '))
    literals_count = dict(Counter(l))

    for clause in cnf:
        literals = clause.split(' ')
        for literal in literals:
            # check if negation of literal in remaining clauses
            if literal[0]=='!':
                literal_negation = literal[1:]
            else:
                literal_negation = '!' + literal
            if literal_negation not in literals_count.keys():
            # if  literals_count[litera]==1 and len(is_literal_in_cnf(literal=literal_negation, cnf=cnf)) == 0:
                print(f"Pure literal {literal} found. Negation of this literal is not present in remaining clauses")
                return literal
    print(f"No pure literal exit, continuing ...")
    return None

In [78]:
cnf_ = ['!V_B', 'Q_R Q_G', '!Q_R !SA_R', '!Q_G !SA_G', 'SA_R SA_G', '!SA_R !Q_R', '!SA_R !V_R', '!SA_R !WA_R', '!SA_G !Q_G', '!SA_G !V_G', '!SA_G !WA_G', 'V_R V_G V_B', '!V_R !SA_R', '!V_G !SA_G', '!V_B', 'WA_R WA_G WA_B', '!WA_R !SA_R', '!WA_G !SA_G']

In [79]:
get_next_pureliteral(cnf_)

Retrieving next pure literal from cnf ...
Pure literal WA_B found. Negation of this literal is not present in remaining clauses


'WA_B'

In [ ]:
is_literal_in_cnf(literal=literal_negation, cnf=cnf)

In [76]:
def get_true_assignments(filename):
    lines = read_input(filename=filename)
    true_assignments = [l.strip('\n') for l in lines if l != '\n']  #remove extra spaces from input
    return true_assignments

In [77]:
get_true_assignments("oz.3.out")

['NSW = Blue',
 'NT = Blue',
 'Q = Green',
 'SA = Red',
 'T = Green',
 'V = Green',
 'WA = Green']

In [ ]:
def check_wrong_assignments(computed_assignments, true_assignments):
    print([i for i in true_assignments if i not in computed_assignments])

In [82]:
cnf = ['NSW_R NSW_G NSW_B', 'NT_R NT_G NT_B', 'Q_R Q_G Q_B', 'SA_R SA_G SA_B', 'T_R T_G T_B', 'V_R V_G V_B', 'WA_R WA_G WA_B', '!NSW_R !Q_R', '!NSW_R !SA_R', '!NSW_R !V_R', '!NSW_G !Q_G', '!NSW_G !SA_G', '!NSW_G !V_G', '!NSW_B !Q_B', '!NSW_B !SA_B', '!NSW_B !V_B', '!NT_R !Q_R', '!NT_R !SA_R', '!NT_R !WA_R', '!NT_G !Q_G', '!NT_G !SA_G', '!NT_G !WA_G', '!NT_B !Q_B', '!NT_B !SA_B', '!NT_B !WA_B', '!Q_R !NSW_R', '!Q_R !NT_R', '!Q_R !SA_R', '!Q_G !NSW_G', '!Q_G !NT_G', '!Q_G !SA_G', '!Q_B !NSW_B', '!Q_B !NT_B', '!Q_B !SA_B', '!SA_R !NSW_R', '!SA_R !NT_R', '!SA_R !Q_R', '!SA_R !V_R', '!SA_R !WA_R', '!SA_G !NSW_G', '!SA_G !NT_G', '!SA_G !Q_G', '!SA_G !V_G', '!SA_G !WA_G', '!SA_B !NSW_B', '!SA_B !NT_B', '!SA_B !Q_B', '!SA_B !V_B', '!SA_B !WA_B', '!V_R !NSW_R', '!V_R !SA_R', '!V_G !NSW_G', '!V_G !SA_G', '!V_B !NSW_B', '!V_B !SA_B', '!WA_R !NT_R', '!WA_R !SA_R', '!WA_G !NT_G', '!WA_G !SA_G', '!WA_B !NT_B', '!WA_B !SA_B']


In [83]:
for i in cnf:
    print(i)

NSW_R NSW_G NSW_B
NT_R NT_G NT_B
Q_R Q_G Q_B
SA_R SA_G SA_B
T_R T_G T_B
V_R V_G V_B
WA_R WA_G WA_B
!NSW_R !Q_R
!NSW_R !SA_R
!NSW_R !V_R
!NSW_G !Q_G
!NSW_G !SA_G
!NSW_G !V_G
!NSW_B !Q_B
!NSW_B !SA_B
!NSW_B !V_B
!NT_R !Q_R
!NT_R !SA_R
!NT_R !WA_R
!NT_G !Q_G
!NT_G !SA_G
!NT_G !WA_G
!NT_B !Q_B
!NT_B !SA_B
!NT_B !WA_B
!Q_R !NSW_R
!Q_R !NT_R
!Q_R !SA_R
!Q_G !NSW_G
!Q_G !NT_G
!Q_G !SA_G
!Q_B !NSW_B
!Q_B !NT_B
!Q_B !SA_B
!SA_R !NSW_R
!SA_R !NT_R
!SA_R !Q_R
!SA_R !V_R
!SA_R !WA_R
!SA_G !NSW_G
!SA_G !NT_G
!SA_G !Q_G
!SA_G !V_G
!SA_G !WA_G
!SA_B !NSW_B
!SA_B !NT_B
!SA_B !Q_B
!SA_B !V_B
!SA_B !WA_B
!V_R !NSW_R
!V_R !SA_R
!V_G !NSW_G
!V_G !SA_G
!V_B !NSW_B
!V_B !SA_B
!WA_R !NT_R
!WA_R !SA_R
!WA_G !NT_G
!WA_G !SA_G
!WA_B !NT_B
!WA_B !SA_B
